# Predicting Shipping Times with Neural Networks

This neural network attempts to predict shipping time (categorized into 3 categories). It is able to predict the right shipping time category with an accuracy of ~70%. 

**Input Features:** \\
"Region", "Ship Mode", "Segment", "Sub-Category" (one-hot encoded)

**Input Shape**: \\
(n, 31)

**Output Shape**: \\
(n, 1)

Colab Link: https://colab.research.google.com/drive/1O35SMsc0DG3OLM9_ViPVXKLywIfuZkyY?usp=sharing

In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import datetime as dt
from tensorflow import keras
from keras import backend as K

In [42]:
url = 'https://raw.githubusercontent.com/dali-lab/data_challenge/main/Sample%20-%20Superstore.csv'
df = pd.read_csv(url, encoding="ISO-8859-1")
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])
df["Shipping Time"] = df["Ship Date"] - df["Order Date"]
features = ["Region", "Ship Mode", "Segment", "Sub-Category"]
input = df[features]
output = df["Shipping Time"].apply(lambda x: x.days // 3) # categories: Fast/0 (0-2), Medium/1 (3-5), Slow/2 (6-7)

In [43]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(input)
encoded_categories = ohe.categories_
x = ohe.transform(input) # all features are one-hot encoded
x_train, x_test, y_train, y_test = train_test_split(x, output)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train.to_numpy())).shuffle(128).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test.to_numpy())).shuffle(128).batch(64)
a = x_train.shape[1] # size of input layer

In [45]:
model = keras.Sequential([
    keras.layers.Input(shape=(a,)),
    keras.layers.Dense(units=17, activation='relu'),
    keras.layers.Dense(units=12, activation='relu'),
    keras.layers.Dense(units=8, activation='relu'),
    keras.layers.Dense(units=3, activation='softmax')
])
model.compile(optimizer='Adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(
    train_dataset.repeat(), 
    epochs=10, 
    steps_per_epoch=500,
    validation_data=test_dataset.repeat(), 
    validation_steps=2
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


500/500 [==============================] - 3s 4ms/step - loss: 0.7283 - accuracy: 0.6267 - val_loss: 0.5833 - val_accuracy: 0.7266
Epoch 2/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6082 - accuracy: 0.6752 - val_loss: 0.5944 - val_accuracy: 0.7109
Epoch 3/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6021 - accuracy: 0.6784 - val_loss: 0.6005 - val_accuracy: 0.7109
Epoch 4/10
500/500 [==============================] - 1s 2ms/step - loss: 0.6015 - accuracy: 0.6763 - val_loss: 0.5959 - val_accuracy: 0.6797
Epoch 5/10
500/500 [==============================] - 1s 2ms/step - loss: 0.5992 - accuracy: 0.6779 - val_loss: 0.5941 - val_accuracy: 0.7031
Epoch 6/10
500/500 [==============================] - 1s 2ms/step - loss: 0.5973 - accuracy: 0.6797 - val_loss: 0.6168 - val_accuracy: 0.6719
Epoch 7/10
500/500 [==============================] - 1s 2ms/step - loss: 0.5965 - accuracy: 0.6780 - val_loss: 0.5997 - val_accuracy: 0.7188
Epoch 8/10
500/50